In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import csv

# Spotify API credentials (replace with your own credentials)
client_id = '9ec5d85af2cc42d498f9a6b3d6b525e3'
client_secret = '7566bb1c94c947d3adff833dd8c1b3f0'

# Initialize Spotipy client with credentials
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


def get_artist_id(artist_name):
    # Search for the artist
    results = sp.search(q='artist:' + artist_name, type='artist')
    if results['artists']['items']:
        # Return the ID of the first artist found
        return results['artists']['items'][0]['id']
    else:
        return None


def get_recent_tracks(artist_id):
    # Get the artist's albums
    albums = sp.artist_albums(artist_id, album_type='album', limit=10)  # Limiting to 10 recent albums

    # Collect tracks from the latest albums
    track_details = []
    for album in albums['items']:
        tracks = sp.album_tracks(album['id'])['items']
        for track in tracks:
            track_info = sp.track(track['id'])
            track_details.append({'name': track_info['name'], 'popularity': track_info['popularity'],
                                  'duration_min': track_info['duration_ms'] / 60000})

            if len(track_details) >= 75:
                return track_details

    return track_details


def create_csv(data):
    with open('songs_data.csv', mode='w', newline='', encoding='utf-8') as file:
        fieldnames = ['Song Name', 'Popularity', 'Duration (min)']
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()
        for track in data:
            writer.writerow({'Song Name': track['name'], 'Popularity': track['popularity'],
                             'Duration (min)': track['duration_min']})


def main():
    artist_name = 'Weeknd'  # Replace with the artist name
    artist_id = get_artist_id(artist_name)

    if artist_id:
        recent_tracks = get_recent_tracks(artist_id)
        print(f"Recent tracks for {artist_name} (up to 75 songs):")
        for idx, track in enumerate(recent_tracks):
            print(f"{idx + 1}. {track['name']} - Popularity: {track['popularity']}, Duration: {track['duration_min']} min")

        # Generate CSV file
        create_csv(recent_tracks)
        print("\nCSV file 'songs_data.csv' generated successfully!")

    else:
        print(f"Artist '{artist_name}' not found.")


if __name__ == "__main__":
    main()
